### NLBSE'24 FastText Baseline

In [1]:
RANDOM_SEED = 42
OUTPUT_PATH = 'output/fasttext'

!mkdir -p $OUTPUT_PATH

In [2]:
from datasets import Dataset

ds = Dataset.from_csv({ "train": "data/issues_train.csv", "test": "data/issues_test.csv" })
ds

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['repo', 'created_at', 'label', 'title', 'body'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['repo', 'created_at', 'label', 'title', 'body'],
        num_rows: 1500
    })
})

In [3]:
repos = ds["train"].unique("repo")
print(repos)

['facebook/react', 'tensorflow/tensorflow', 'microsoft/vscode', 'bitcoin/bitcoin', 'opencv/opencv']


In [4]:
ds["train"].to_pandas().groupby(["repo", "label"]).size().unstack(fill_value=0)

label,bug,feature,question
repo,,,
bitcoin/bitcoin,100,100,100
facebook/react,100,100,100
microsoft/vscode,100,100,100
opencv/opencv,100,100,100
tensorflow/tensorflow,100,100,100


In [5]:
import gensim

def process_dataset(example):

    # concatenate title and body
    text = (example['title'] or "") + " " + (example['body'] or "")

    # escape fasttext special sequences
    text = text.replace("__label__", "")

    # remove consecutive whitespace characters and convert tabs to spaces
    text = gensim.parsing.preprocessing.strip_multiple_whitespaces(text)
    
    example['text'] = text

    return example

ds = ds.shuffle(seed=RANDOM_SEED)
ds = ds.map(process_dataset)
ds = ds.select_columns(['repo', 'label','text'])
ds

DatasetDict({
    train: Dataset({
        features: ['repo', 'label', 'text'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['repo', 'label', 'text'],
        num_rows: 1500
    })
})

In [6]:
import fasttext

def repo_eq(repo: str):
    return lambda example: example['repo'] == repo

def to_fasttext_train_file(dataset, o_path):
    with open(o_path, "w", encoding='utf-8') as o_f:
        for example in dataset:
            o_f.write(f"__label__{example['label']} {example['text']}\n")

references = {}
predictions = {}

for repo in repos:
    train_file = f"{OUTPUT_PATH}/issues-{repo.replace('/','-')}.train"
    model_file = f"{OUTPUT_PATH}/issues-{repo.replace('/','-')}.bin"

    train_set = ds.filter(repo_eq(repo))["train"]

    to_fasttext_train_file(train_set, train_file)

    model = fasttext.train_supervised(input=train_file, epoch=100, seed=RANDOM_SEED)

    model.save_model(model_file)
    
    test_set = ds.filter(repo_eq(repo))["test"]
    
    references[repo] = test_set['label']
    predictions[repo] = model.predict(test_set['text'], k=1)

    predictions[repo] = predictions[repo][0]
    predictions[repo] = [pred[0] for pred in predictions[repo]]
    predictions[repo] = [pred.replace('__label__', '') for pred in predictions[repo]]


Read 0M words
Number of words:  9953
Number of labels: 3
Progress: 100.0% words/sec/thread: 4762628 lr:  0.000000 avg.loss:  0.410777 ETA:   0h 0m 0s
Read 0M words
Number of words:  15399
Number of labels: 3
Progress: 100.0% words/sec/thread: 3794275 lr:  0.000000 avg.loss:  0.576489 ETA:   0h 0m 0s
Read 0M words
Number of words:  7682
Number of labels: 3
Progress: 100.0% words/sec/thread: 3523742 lr:  0.000000 avg.loss:  0.395247 ETA:   0h 0m 0s
Read 0M words
Number of words:  15844
Number of labels: 3
Progress: 100.0% words/sec/thread: 3701482 lr:  0.000000 avg.loss:  0.452791 ETA:   0h 0m 0s
Read 0M words
Number of words:  15863
Number of labels: 3
Progress: 100.0% words/sec/thread: 4439443 lr:  0.000000 avg.loss:  0.559553 ETA:   0h 0m 0s


In [7]:
from sklearn.metrics import classification_report
from numpy import mean

results = {}
metrics = ['precision', 'recall', 'f1-score']
labels = ['bug', 'feature', 'question']

for repo in repos:
  results[repo] = classification_report(references[repo], predictions[repo], digits=4, output_dict=True)
  results[repo]['average'] = results[repo]['weighted avg']
  results[repo] = {label: {metric: results[repo][label][metric] for metric in metrics} for label in labels + ['average']}

results['overall'] = {label: {metric: mean([results[repo][label][metric] for repo in repos]) for metric in metrics} for label in labels + ['average']}


In [8]:
import json
import os

output_file_name = 'results.json'

with open(os.path.join(OUTPUT_PATH, output_file_name), 'w') as fp:
    json.dump(results, fp, indent=2)

print(f"Repository{' '*15}Label     Precision  Recall     F1")
for repo in repos + ['overall']:
  print("-"*63)
  for label in labels + ['average']:
    out = f"{repo:<25}{label:<10}"
    for metric in metrics:
      out += f"{results[repo][label][metric]:<10.4f} "
    print(out)

Repository               Label     Precision  Recall     F1
---------------------------------------------------------------
facebook/react           bug       0.9278     0.9000     0.9137     
facebook/react           feature   0.7451     0.7600     0.7525     
facebook/react           question  0.6931     0.7000     0.6965     
facebook/react           average   0.7887     0.7867     0.7876     
---------------------------------------------------------------
tensorflow/tensorflow    bug       0.7364     0.8100     0.7714     
tensorflow/tensorflow    feature   0.7889     0.7100     0.7474     
tensorflow/tensorflow    question  0.6000     0.6000     0.6000     
tensorflow/tensorflow    average   0.7084     0.7067     0.7063     
---------------------------------------------------------------
microsoft/vscode         bug       0.7640     0.6800     0.7196     
microsoft/vscode         feature   0.6522     0.7500     0.6977     
microsoft/vscode         question  0.7812     0.7500     0